In [108]:
import pandas as pd
from pandas.io.json import json_normalize
import json


eventlog_df = pd.read_json('cmd_psexec_lsa_secrets_dump_2020-10-1903305471.json', lines=True)

In [109]:
eventlog_df.head()

,SourceName,TimeCreated,Hostname,Task,Channel,Message,EventID,FilterRTID,SourcePort,SourceAddress,...,DestinationPort,SourceIsIpv6,SourceHostname,Initiated,QueryName,QueryStatus,QueryResults,StartType,AccountName,ImagePath
0,Microsoft-Windows-Eventlog,2020-10-19 03:30:41.104,WORKSTATION5,104,Security,The audit log was cleared.\r\nSubject:\r\n\tSe...,1102,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Microsoft-Windows-Security-Auditing,2020-10-19 03:30:45.323,WORKSTATION5,12810,Security,The Windows Filtering Platform has permitted a...,5158,0.0,58390.0,0.0.0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Microsoft-Windows-Security-Auditing,2020-10-19 03:30:45.324,WORKSTATION5,12810,Security,The Windows Filtering Platform has permitted a...,5156,69895.0,58390.0,192.168.2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Microsoft-Windows-Security-Auditing,2020-10-19 03:30:46.251,WORKSTATION5,13312,Security,A new process has been created.\r\n\r\nCreator...,4688,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Microsoft-Windows-Security-Auditing,2020-10-19 03:30:46.330,WORKSTATION5,12807,Security,An attempt was made to duplicate a handle to a...,4690,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
#Print rows contained within index 0 to ensure event log has been parsed into the df correctly.
eventlog_df.iloc[0]

SourceName                                      Microsoft-Windows-Eventlog
TimeCreated                                        2020-10-19 03:30:41.104
Hostname                                                      WORKSTATION5
Task                                                                   104
Channel                                                           Security
Message                  The audit log was cleared.\r\nSubject:\r\n\tSe...
EventID                                                               1102
FilterRTID                                                             NaN
SourcePort                                                             NaN
SourceAddress                                                          NaN
Protocol                                                               NaN
ProcessId                                                              NaN
LayerRTID                                                              NaN
LayerName                

In [111]:
#In this example we know that psexec is used. There are a variety of ways to hunt for psexec useage including but not limited to keyword searching in prefetch, hashes etc depending on the forensic data you have at your disposal. 
#In this case I'll search for 7045 events as PsExec will install itself on a service on a destination host with the service file name recorded as PSEXESVC.exe

import pandas as pd

# Select the rows with EventID = 7045 and ServiceName = 'PSEXESVC'
filtered_df = eventlog_df.loc[(eventlog_df['EventID'] == 7045) & (eventlog_df['ServiceName'] == 'PSEXESVC')]

# Iterate through the rows and print the values
for index, row in filtered_df.iterrows():
  print(f'Hostname = {row["Hostname"]}')
  print(f'HostName = {row["TimeCreated"]}')
  print(f'EventID = {row["EventID"]}')
  print(f'Message = {row["Message"]}')
  print(f'ParentCommandLine = {row["ParentCommandLine"]}')
  print(f'CommandLine = {row["CommandLine"]}')
  



Hostname = WORKSTATION5
HostName = 2020-10-19 03:30:46.416
EventID = 7045
Message = A service was installed in the system.

Service Name:  PSEXESVC
Service File Name:  %SystemRoot%\PSEXESVC.exe
Service Type:  user mode service
Service Start Type:  demand start
Service Account:  LocalSystem
ParentCommandLine = nan
CommandLine = nan


In [112]:
#Get a count of each eventId

In [113]:
counts = eventlog_df['EventID'].value_counts()
sort_counts = counts.sort_values()
for value, count in sort_counts.items():
    print(f"EventID: {value} | Occurance: {count}")

EventID: 7045 | Occurance: 1
EventID: 5140 | Occurance: 1
EventID: 5145 | Occurance: 1
EventID: 1102 | Occurance: 1
EventID: 104 | Occurance: 1
EventID: 4697 | Occurance: 1
EventID: 3 | Occurance: 2
EventID: 22 | Occurance: 2
EventID: 5158 | Occurance: 3
EventID: 9 | Occurance: 3
EventID: 5156 | Occurance: 4
EventID: 4688 | Occurance: 4
EventID: 1 | Occurance: 4
EventID: 4689 | Occurance: 4
EventID: 4690 | Occurance: 4
EventID: 4703 | Occurance: 4
EventID: 4656 | Occurance: 4
EventID: 4663 | Occurance: 4
EventID: 5 | Occurance: 4
EventID: 18 | Occurance: 7
EventID: 11 | Occurance: 7
EventID: 17 | Occurance: 7
EventID: 4658 | Occurance: 8
EventID: 12 | Occurance: 14
EventID: 13 | Occurance: 23
EventID: 10 | Occurance: 38
EventID: 7 | Occurance: 130


In [130]:
events_less_than_10 = counts[counts <10].index

stacked_df = eventlog_df[eventlog_df['EventID'].isin(events_less_than_10)]

for _, row in stacked_df.iterrows():
    print(f"EventID: {row['EventID']} - Message: {row['Message']}")


EventID: 1102 - Message: The audit log was cleared.
Subject:
	Security ID:	S-1-5-21-3940915590-64593676-1414006259-500
	Account Name:	wardog
	Domain Name:	WORKSTATION5
	Logon ID:	0xC61D9
EventID: 5158 - Message: The Windows Filtering Platform has permitted a bind to a local port.

Application Information:
	Process ID:		3304
	Application Name:	\device\harddiskvolume2\windowsazure\guestagent_2.7.41491.993_2020-10-08_063613\waappagent.exe

Network Information:
	Source Address:		0.0.0.0
	Source Port:		58390
	Protocol:		6

Filter Information:
	Filter Run-Time ID:	0
	Layer Name:		Resource Assignment
	Layer Run-Time ID:	36
EventID: 5156 - Message: The Windows Filtering Platform has permitted a connection.

Application Information:
	Process ID:		3304
	Application Name:	\device\harddiskvolume2\windowsazure\guestagent_2.7.41491.993_2020-10-08_063613\waappagent.exe

Network Information:
	Direction:		Outbound
	Source Address:		192.168.2.5
	Source Port:		58390
	Destination Address:	168.63.129.16
	D

In [131]:
#get stacked event_logs into timeline
writer = pd.ExcelWriter('timeline.xlsx', engine='openpyxl', mode='a')

stacked_df[['Hostname', 'TimeCreated', 'EventID', 'Message', 'ParentCommandLine', 'CommandLine']].to_excel(writer, index=False, header=False, sheet_name='Stacked_Events')

writer.save()

ValueError: Sheet 'Stacked_Events' already exists and if_sheet_exists is set to 'error'.

In [116]:
# Stack CommandLines

In [117]:
commandLineCounts = eventlog_df['CommandLine'].value_counts() 
print(commandLineCounts)

PsExec.exe  -accepteula -s reg save HKLM\security\policy\secrets C:\Users\wardog\AppData\Local\Temp\secrets    2
C:\windows\PSEXESVC.exe                                                                                        2
"reg" save HKLM\security\policy\secrets C:\Users\wardog\AppData\Local\Temp\secrets                             2
\??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1                                                        2
Name: CommandLine, dtype: int64


In [118]:
filteredForCommandLines = eventlog_df[eventlog_df['CommandLine'].notnull()]

for index, row in filteredForCommandLines.iterrows():
    hostname = row['Hostname']
    time_created = row['TimeCreated']
    event_id = row['EventID']
    message = row['Message']
    parent_command_line = row['ParentCommandLine']
    command_line = row['CommandLine']
    print(f'Hostname: {hostname}, TimeCreated: {time_created}, ParentCommandLine: {parent_command_line}, CommandLine: {command_line}')


Hostname: WORKSTATION5, TimeCreated: 2020-10-19 03:30:46.251, ParentCommandLine: nan, CommandLine: PsExec.exe  -accepteula -s reg save HKLM\security\policy\secrets C:\Users\wardog\AppData\Local\Temp\secrets
Hostname: WORKSTATION5, TimeCreated: 2020-10-19 03:30:46.435, ParentCommandLine: nan, CommandLine: C:\windows\PSEXESVC.exe
Hostname: WORKSTATION5, TimeCreated: 2020-10-19 03:30:46.660, ParentCommandLine: nan, CommandLine: "reg" save HKLM\security\policy\secrets C:\Users\wardog\AppData\Local\Temp\secrets
Hostname: WORKSTATION5, TimeCreated: 2020-10-19 03:30:46.673, ParentCommandLine: nan, CommandLine: \??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1
Hostname: WORKSTATION5, TimeCreated: 2020-10-19 03:30:46.257, ParentCommandLine: "C:\windows\system32\cmd.exe" , CommandLine: PsExec.exe  -accepteula -s reg save HKLM\security\policy\secrets C:\Users\wardog\AppData\Local\Temp\secrets
Hostname: WORKSTATION5, TimeCreated: 2020-10-19 03:30:46.438, ParentCommandLine: C:\windows\system3

In [134]:
#Get these into the timeline
writer = pd.ExcelWriter('timeline.xlsx', engine='openpyxl', mode='a')

filteredForCommandLines = eventlog_df[eventlog_df['CommandLine'].notnull()]

filteredForCommandLines[['Hostname', 'TimeCreated', 'EventID', 'Message', 'ParentCommandLine', 'CommandLine']].to_excel(writer, index=False, header=False, sheet_name="Stacked_CommandLines")

writer.save()

In [137]:
#get the entire dataframe into the spreadsheet.
writer = pd.ExcelWriter('timeline.xlsx', engine='openpyxl', mode='a')

eventlog_df.to_excel(writer, index=False, header=False, sheet_name="DataFrame")

writer.save()